# Client

httpx.Client() 类似于 requests.Session()

- Client 实例使用 HTTP 连接池。当向同一主机发出多个请求时， Client 将重用底层 TCP 连接
- 跨请求的 Cookie 持久性。
- 支持 HTTP proxies


In [2]:
import httpx

## Event Hooks


### log


In [17]:
def log_request(request):
    print(
        f"Request event hook: {request.method} {request.url} {request.headers} - Waiting for response"
    )


def log_response(response):
    request = response.request
    print(
        f"Response event hook: {request.method} {request.url} - Status {response.status_code}"
    )


event_client = httpx.Client(
    event_hooks={"request": [log_request], "response": [log_response]}
)

In [18]:
r = event_client.get("https://www.example.org/", headers={"k1": "v1"})
print(f"-----------resp:{event_client}")

Request event hook: GET https://www.example.org/ Headers({'host': 'www.example.org', 'accept': '*/*', 'accept-encoding': 'gzip, deflate', 'connection': 'keep-alive', 'user-agent': 'python-httpx/0.25.2', 'k1': 'v1'}) - Waiting for response
Response event hook: GET https://www.example.org/ - Status 200
-----------resp:<httpx.Client object at 0x0000017667307250>


### change request


In [5]:
def add_timestamp(request):
    request.headers["x-request-timestamp"] = "now"


change_request_client = httpx.Client(event_hooks={"request": [add_timestamp]})

## proxies


### base


In [ ]:
with httpx.Client(proxies="http://localhost:7890") as client:
    r = client.get("https://www.google.com/")
    print(r.text)

### mounts


In [ ]:
from httpx import Proxy


proxy_mounts = {

    "http://": httpx.HTTPTransport(proxy=Proxy(url="http://localhost:7890")),

    "https://": httpx.HTTPTransport(proxy=Proxy(url="http://localhost:7890")),

}



with httpx.Client(mounts=proxy_mounts) as client:

    r = client.get("https://www.google.com/")
    print(r.text)

## timeout

可能会发生四种不同类型的超时。这些是连接、读取、写入和池超时。


### base


In [ ]:
with httpx.Client(timeout=10) as client:
    r = client.get("https://www.google.com/")
    print(r.text)

### connect


In [ ]:
timeout = httpx.Timeout(10.0, connect=60.0)
client = httpx.Client(timeout=timeout)

### read


In [14]:
timeout = httpx.Timeout(10.0, read=60.0)
client = httpx.Client(timeout=timeout)

### write


In [ ]:
timeout = httpx.Timeout(10.0, write=60.0)
client = httpx.Client(timeout=timeout)

### pool


In [ ]:
timeout = httpx.Timeout(10.0, pool=60.0)
client = httpx.Client(timeout=timeout)

## Monitoring download progress


In [24]:
import tempfile

import httpx
from tqdm import tqdm

with tempfile.NamedTemporaryFile() as download_file:
    with httpx.stream("GET", "https://speed.hetzner.de/100MB.bin") as response:
        total = int(response.headers["Content-Length"])
        with tqdm(
            total=total, unit_scale=True, unit_divisor=1024, unit="B"
        ) as progress:
            num_bytes_downloaded = response.num_bytes_downloaded
            for chunk in response.iter_bytes():
                download_file.write(chunk)
                progress.update(response.num_bytes_downloaded - num_bytes_downloaded)
                num_bytes_downloaded = response.num_bytes_downloaded

100%|██████████| 190/190 [00:00<00:00, 184kB/s]


## Customizing authentication


### base


In [20]:
class MyCustomAuth(httpx.Auth):
    def __init__(self, token):
        self.token = token

    def auth_flow(self, request):
        request.headers["X-Authentication"] = self.token
        yield request


client = httpx.Client(auth=MyCustomAuth("token"))
r = client.get("https://www.google.com/")
r

<Response [200 OK]>

### if 401


In [25]:
class MyCustomAuth(httpx.Auth):
    def __init__(self, token):
        self.token = token

    def auth_flow(self, request):
        response = yield request
        if response.status_code == 401:
            request.headers["X-Authentication"] = self.token
            yield request

### asyncio auto


In [23]:
import asyncio
import threading
import httpx


class MyAsyncCustomAuth(httpx.Auth):
    def __init__(self):
        self._sync_lock = threading.RLock()
        self._async_lock = asyncio.Lock()

    def sync_get_token(self):
        with self._sync_lock:
            ...

    def sync_auth_flow(self, request):
        token = self.sync_get_token()
        request.headers["Authorization"] = f"Token {token}"
        yield request

    async def async_get_token(self):
        async with self._async_lock:
            ...

    async def async_auth_flow(self, request):
        token = await self.async_get_token()
        request.headers["Authorization"] = f"Token {token}"
        yield request


client = httpx.AsyncClient(auth=MyAsyncCustomAuth())
r = await client.get("https://www.google.com/")
r

<Response [200 OK]>